In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="/content",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="/content",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting /content/FashionMNIST/raw/train-images-idx3-ubyte.gz to /content/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting /content/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting /content/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting /content/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/FashionMNIST/raw



In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [5]:
# 손실 함수를 초기화합니다.
loss_fn = nn.CrossEntropyLoss()

In [7]:
# optimizer의 매개변수로는 train하고자 하는 model의 parameter를 전달한다.
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        # loss.backward()를 해줄 때 계속해서 gradient를 더해주게 된다.
        # 이 때문에 optimizer.zero_grad()를 통해 
        # loop 반복시마다 gradient를 0으로 초기화 해줘야 한다.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301011  [    0/60000]
loss: 2.291157  [ 6400/60000]
loss: 2.275157  [12800/60000]
loss: 2.270615  [19200/60000]
loss: 2.248794  [25600/60000]
loss: 2.236140  [32000/60000]
loss: 2.235340  [38400/60000]
loss: 2.210527  [44800/60000]
loss: 2.208877  [51200/60000]
loss: 2.170183  [57600/60000]
Test Error: 
 Accuracy: 43.8%, Avg loss: 2.166619 

Epoch 2
-------------------------------
loss: 2.177232  [    0/60000]
loss: 2.164023  [ 6400/60000]
loss: 2.111938  [12800/60000]
loss: 2.123733  [19200/60000]
loss: 2.075428  [25600/60000]
loss: 2.028196  [32000/60000]
loss: 2.047549  [38400/60000]
loss: 1.975541  [44800/60000]
loss: 1.983668  [51200/60000]
loss: 1.907378  [57600/60000]
Test Error: 
 Accuracy: 49.7%, Avg loss: 1.902568 

Epoch 3
-------------------------------
loss: 1.936853  [    0/60000]
loss: 1.901183  [ 6400/60000]
loss: 1.788509  [12800/60000]
loss: 1.828191  [19200/60000]
loss: 1.727025  [25600/60000]
loss: 1.671433  [32000/600